In [1]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect


In [2]:
try:
    __confing.execute_notebook('notebooks/data_extract.ipynb')
except:
    pass

In [3]:
import pandas as pd
import numpy as np

from src import value_filler as vf
from src import feature_selection as fs
from src import data_procces as dp

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from src import evaluation

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,roc_auc_score,confusion_matrix
from sklearn.model_selection import KFold

In [4]:
def feature_importance(model, X: pd.DataFrame, y: pd.Series, n: int):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    feature_names_original = list(X.columns)
    
    target_name = y.name  # Obtener el nombre de la columna objetivo desde la Serie y
    
    if target_name in feature_names_original:
        feature_names_original.remove(target_name)
    
    max_importance = list(indices[:n])
    
    nueva_lista = [feature_names_original[i] for i in max_importance]
    
    return nueva_lista


In [28]:
n='a'
df=pd.read_csv(r'data\custom\all_waves.csv')
target=f'p{n}hosp1y'
drop=[f'p{n}hspnit1y','paoophos1y','paoophosf1y'] # si la wave3 falla quitar estas columnas , 'r3bpsft', 'r3bpref'

In [29]:
# Drop
df = df.drop(drop, axis=1)

df = df.dropna(subset=[target]) 

porcentaje_nulos = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (porcentaje_nulos <= 0.3)]

In [30]:
# fill
y = df[target]
X1 = df.drop(target,axis=1)
X1 = fs.fast_fill_2(X1)

In [31]:
# feature selection
feature_selected=['padoctor1y','paurina2y','pacholst','pabreast','paprost','papapsm','pamammog','pacage','padrinkcr','padrinkbd','padrinkb','paheight','paweight','pabmi','papaina','papainlv','papainfr','pafatigue','pawheeze','pabreath_m','paswell','pahearaid','pahearing','pasight','paglasses','palunglmt_m','pahrtatlmt','pastroklmt','paarthlmt','palowermoba','pauppermoba','pagrossaa','palgmusaa','pamobilaa','paclims','pasit','paarms','palift','pastoop','pachair','pameals','pashop','pamoney','pameds','pabedhlp','pabed','patoilt','paeat','pabath','pawalkr','padress','pahipcomp','pafall','pafallinj','pamhip','pahipe_m','parxarthr','parxstrok','parxhrtat','pacncrothr','pacncrmeds','pacncrradn','pacncrsurg','pacncrchem','parxdiab','parxdiabo','parxhibp','parechrtatt','parifaany','parfaany','parfcaren','pararcare','padresshlp', 'pawalkhlp', 'pabathehlp', 'paeathlp', 'pabedhlp', 'patoilethlp', 'pamealhlp', 'pashophlp', 'pamedhlp', 'pamoneyhlp','pahibpe','pagender','padiabe','pacancre','palunglmt_m','parxlung_m','pahrtatte','pastroke','paarthre','parifaany','parafaany','pahigov','pasmokev','pasmoken','paoangry','paosleep','paodngr','paodngr','paopace','paoplot','paoalchl','pawthh','paagey','pamomage','padadage','paprmem','parjudg','parorgnz']
X=X1[feature_selected]
print(len(feature_selected))

110


In [32]:
# split
X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=40)
X_test_o, y_test_o = X_test, y_test

In [33]:
# resample 
X_train, y_train = dp.apply_resample(X_train, y_train,v=1.20)
X_test, y_test = dp.apply_resample(X_test, y_test,v=1.20)

In [34]:
# smote
X_train, y_train = dp.apply_smote(X_train, y_train)

In [35]:
# scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

In [38]:
import xgboost as xgb

from sklearn.model_selection import KFold

def train_xgboost_with_kfold(features, labels, model):
    k = 15
    kf = KFold(n_splits=k)
    
    for train_index, test_index in kf.split(features):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        
        # Entrenar el modelo XGBoost
        model.fit(X_train, y_train)
    
    return model

model_k = xgb.XGBClassifier()
model_k= train_xgboost_with_kfold(X_train, y_train,model_k)

y_pred_proba = model_k.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f'AUC={auc}')
print(y_test.value_counts())

print('')
print('######################################################')
print('')

y_pred_proba = model_k.predict_proba(X_test_o)[:, 1]
auc = roc_auc_score(y_test_o, y_pred_proba)
print(f'AUC={auc}')
print(y_test_o.value_counts())

AUC=0.9659245954617675
pahosp1y
0.0    3528
1.0    2940
Name: count, dtype: int64

######################################################

AUC=0.8546781003836462
pahosp1y
1.0    2940
0.0    1798
Name: count, dtype: int64


In [40]:
# raise ValueError('comentar el error')
li = feature_importance(model_k, X, y, n=70)
print(li)

['parfcaren', 'paprmem', 'parfaany', 'padoctor1y', 'pacholst', 'pagrossaa', 'pahrtatte', 'pabathehlp', 'parxdiab', 'pacncrchem', 'pacancre', 'paoangry', 'parxhrtat', 'pastoop', 'pafallinj', 'pastroklmt', 'pashophlp', 'parxstrok', 'pabedhlp', 'papainfr', 'parechrtatt', 'pawalkhlp', 'pacncrmeds', 'palunglmt_m', 'padiabe', 'padrinkcr', 'pasight', 'padresshlp', 'paarthlmt', 'pahearaid', 'paurina2y', 'parxhibp', 'pamammog', 'pasmoken', 'padrinkbd', 'pawheeze', 'palift', 'pastroke', 'paswell', 'pagender', 'pameals', 'paagey', 'parxlung_m', 'pamoney', 'pacage', 'pashop', 'pahearing', 'pahipe_m', 'paprost', 'paheight', 'pabmi', 'pabed', 'palgmusaa', 'parafaany', 'padadage', 'pauppermoba', 'paarms', 'pamoneyhlp', 'paglasses', 'papapsm', 'parifaany', 'pameds', 'patoilt', 'paclims', 'pawthh', 'padrinkb', 'pahipcomp', 'pachair', 'parxarthr', 'palowermoba']
